In [106]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

# Census API Key
from config import census_key
c = Census(census_key, year=2016)

In [107]:
#B05007 - PLACE OF BIRTH BY YEAR OF ENTRY BY CITIZENSHIP STATUS FOR THE FOREIGN-BORN POPULATION - 2016 ACS 5-year estimates

# Run Census Search to retrieve data on all zip codes (2016 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

census_data = c.acs5.get(("NAME", 
                          # Summary wealth stats (May drop later)
                          "B01003_001E", 'B01002_001E', 'B19013_001E', 'B19301_001E', 'B17001_002E',
                          # Total Foreign Born
                          "B05007_001E",
                          #Entered 2010 or Later
                          "B05007_002E", "B05007_003E","B05007_004E",
                          #Entered 2000 to 2009
                           "B05007_005E", "B05007_006E", "B05007_007E",
                          #Entered 1990 to 1999
                          "B05007_008E", "B05007_009E", "B05007_010E",
                          #Earlier than 1990
                         "B05007_011E", "B05007_012E","B05007_013E"), 
                         geo={'for': 'place:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
census_pd.head(1)

,B01002_001E,B01003_001E,B05007_001E,B05007_002E,B05007_003E,B05007_004E,B05007_005E,B05007_006E,B05007_007E,B05007_008E,...,B05007_010E,B05007_011E,B05007_012E,B05007_013E,B17001_002E,B19013_001E,B19301_001E,NAME,place,state
0,32.9,151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,41.0,-666666666.0,8940.0,"Abanda CDP, Alabama",00100,01


In [108]:
# Column renaming

# Natu = Naturalized (Became a citizen)
# NCit = Not a Citizen
# Total = Natu + NCit
# Total_ForeignBorn = total from any year

census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B05007_001E": "Total_ForeignBorn",
                                      "B05007_002E": "Total_2010",
                                      "B05007_003E": "Natu_2010",
                                      "B05007_004E": "NCit_2010",
                                      "B05007_005E": "Total_2000",
                                      "B05007_006E": "Natu_2000",
                                      "B05007_007E": "NCit_2000",
                                      "B05007_008E": "Total_1990",
                                      "B05007_009E": "Natu_1990",
                                      "B05007_010E": "NCit_1990",
                                      "B05007_011E": "Total_Early90",
                                      "B05007_012E": "Natu_Early90",
                                      "B05007_013E": "NCit_Early90",
                                      "NAME": "combined_name",
                                      "place": "county_fips",
                                      "state": "state_fips"
                                     })
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Foreign born Rate (Poverty Count / Population)
census_pd["ForeignBorn_Rate"] = 100 * \
    census_pd["Total_ForeignBorn"].astype(
        int) / census_pd["Population"].astype(int)

    
# Add in Combined FIPS
census_pd['combined_fips'] = census_pd['state_fips'].astype(str) + census_pd['county_fips']

# Separate State and County into Columns
census_pd['place_name'], census_pd['state_name'] = census_pd['combined_name'].str.split(', ', 1).str

# Drop Puerto Rico
#print(census_pd.count())
census_pd = census_pd[census_pd['state_name'] != 'Puerto Rico']
#print(census_pd.count())

# Keep leading zeros
census_pd['combined_fips'] = census_pd['combined_fips'].apply(str)
census_pd['state_fips'] = census_pd['state_fips'].apply(str)
census_pd['county_fips'] = census_pd['county_fips'].apply(str)
census_pd = census_pd[census_pd.place_name.str.contains(" CDP") == False]

strips = (' town', " city", ' City', " CDP" )
#for x in strips:

census_pd['place_name']= census_pd.loc[:, 'place_name'].replace(regex=True, to_replace=" town", value="")
census_pd['place_name']=census_pd.loc[:, 'place_name'].replace(regex=True, to_replace=" city", value="")
census_pd['place_name']=census_pd.loc[:, 'place_name'].replace(regex=True, to_replace=" CDP", value="")

for (idx, row) in census_pd.iterrows():
    state_fip = (row.loc['state_fips'])
    state_abbr = states.lookup(state_fip).abbr
    census_pd.at[idx, 'state_abbr'] = state_abbr         

census_pd['name.x'] = census_pd['place_name'] + " " + census_pd['state_abbr']
print(census_pd.count())

# Reorder
cols = census_pd.columns.tolist()
cols.insert(0, cols.pop(cols.index('combined_fips')))
cols.insert(1, cols.pop(cols.index('place_name')))
cols.insert(2, cols.pop(cols.index('state_name')))
cols.insert(3, cols.pop(cols.index('name.x')))
census_pd = census_pd.reindex(columns= cols)

# Visualize
print(len(census_pd))
print(census_pd.dtypes)

census_pd = census_pd.rename(columns={"county_fips": "place_fips"})
census_pd.head(10)

Median Age           19534
Population           19534
Total_ForeignBorn    19534
Total_2010           19534
Natu_2010            19534
NCit_2010            19534
Total_2000           19534
Natu_2000            19534
NCit_2000            19534
Total_1990           19534
Natu_1990            19534
NCit_1990            19534
Total_Early90        19534
Natu_Early90         19534
NCit_Early90         19534
Poverty Count        19534
Household Income     19534
Per Capita Income    19499
combined_name        19534
county_fips          19534
state_fips           19534
Poverty Rate         19516
ForeignBorn_Rate     19516
combined_fips        19534
place_name           19534
state_name           19534
state_abbr           19534
name.x               19534
dtype: int64
19534
combined_fips         object
place_name            object
state_name            object
name.x                object
Median Age           float64
Population           float64
Total_ForeignBorn    float64
Total_2010           f

,combined_fips,place_name,state_name,name.x,Median Age,Population,Total_ForeignBorn,Total_2010,Natu_2010,NCit_2010,...,NCit_Early90,Poverty Count,Household Income,Per Capita Income,combined_name,place_fips,state_fips,Poverty Rate,ForeignBorn_Rate,state_abbr
1,0100124,Abbeville,Alabama,Abbeville AL,50.4,2627.0,12.0,0.0,0.0,0.0,...,0.0,601.0,28148.0,20756.0,"Abbeville city, Alabama",00124,01,22.877807,0.456795,AL
2,0100460,Adamsville,Alabama,Adamsville AL,42.0,4422.0,42.0,0.0,0.0,0.0,...,34.0,682.0,44833.0,23732.0,"Adamsville city, Alabama",00460,01,15.422886,0.949796,AL
3,0100484,Addison,Alabama,Addison AL,46.2,757.0,0.0,0.0,0.0,0.0,...,0.0,220.0,34063.0,21097.0,"Addison town, Alabama",00484,01,29.062087,0.000000,AL
4,0100676,Akron,Alabama,Akron AL,43.9,252.0,0.0,0.0,0.0,0.0,...,0.0,110.0,17344.0,10092.0,"Akron town, Alabama",00676,01,43.650794,0.000000,AL
5,0100820,Alabaster,Alabama,Alabaster AL,37.2,32269.0,2193.0,446.0,19.0,427.0,...,149.0,3262.0,73325.0,28239.0,"Alabaster city, Alabama",00820,01,10.108773,6.795996,AL
6,0100988,Albertville,Alabama,Albertville AL,33.4,21434.0,2992.0,207.0,0.0,207.0,...,320.0,5970.0,32042.0,18023.0,"Albertville city, Alabama",00988,01,27.852944,13.959130,AL
7,0101132,Alexander City,Alabama,Alexander City AL,39.5,14859.0,495.0,0.0,0.0,0.0,...,8.0,4847.0,30442.0,17833.0,"Alexander City city, Alabama",01132,01,32.619961,3.331314,AL
9,0101228,Aliceville,Alabama,Aliceville AL,32.7,3218.0,6.0,0.0,0.0,0.0,...,0.0,1296.0,19926.0,11053.0,"Aliceville city, Alabama",01228,01,40.273462,0.186451,AL
10,0101396,Allgood,Alabama,Allgood AL,30.1,663.0,173.0,7.0,0.0,7.0,...,3.0,185.0,40817.0,16140.0,"Allgood town, Alabama",01396,01,27.903469,26.093514,AL
11,0101660,Altoona,Alabama,Altoona AL,43.5,768.0,5.0,0.0,0.0,0.0,...,0.0,180.0,28672.0,16783.0,"Altoona town, Alabama",01660,01,23.437500,0.651042,AL


In [109]:
census_pd.to_csv('census_immig_city.csv')